Для нахождения будем использовать файл `./files/67-blast.txt`.  
Заметим, что каждый описание видов каждого запроса начинается со строки `Description`.  
Далее идет описание хитов с информацией о виде.  
Для идентификации вида будем использовать `Taxid`. Таксономы начинаются в колонке 98 (считая с нуля). 
Во время тестирования скрипта была обнаружена интересная ошибка (скорее всего самого `NCBI`). Почему-то у некоторых видов (а именно, `Stenella coeruleoalba` и `Rangifer tarandus platyrhyncus`) `Taxid` равнялся 0 (также отсутствовал `Scientific name` и `Common name`). Чтобы не усложнять скрипт, для работы `Taxid` у данных видов был пропатчен вручную.

In [74]:
taxa_start_col = 98

def parse_top10_taxa(input_file: str) -> dict[int, int]:
    taxa_rank = {}

    with open(input_file, "r") as f:
        parsing_hit_taxa = False
        taxa_count = 0
        for line in f.readlines():
            if line.startswith("Description"):
                parsing_hit_taxa = True
                continue
            if parsing_hit_taxa:
                if line == "\n":
                    parsing_hit_taxa = False
                    continue

                taxon = int(line[taxa_start_col:line.find(" ", taxa_start_col)])
                if taxon in taxa_rank.keys():
                    taxa_rank[taxon] += 1
                else:
                    taxa_rank[taxon] = 1

                taxa_count += 1
                
                # Ограничиваемся топ-10.
                if taxa_count == 10:
                    parsing_hit_taxa = False

    return taxa_rank

In [80]:
import requests
import xml.etree.ElementTree as ET
import time

In [83]:
species_name_fetch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=taxonomy&id={taxid}"

"""
Так как Scientific Name и Name могут быть сокращены, будем использовать простой api entrez.
"""
def get_species_name_from_taxid(taxid: int) -> str:
    """
    Entrez api от NCBI имеет лимит запросов в 3 запроса в секунду (для запросов без указания api key).  
    Поэтому будем делать все в цикле while с задержкой в секунду.
    """
    while True:
        r = requests.get(species_name_fetch_url.format(taxid=taxid))
        if r.status_code != 200:
            time.sleep(1)
            continue
        
        xml_root = ET.fromstring(r.text)
        return xml_root[0][1].text

In [84]:
taxa_rank = parse_top10_taxa("./files/67-blast.txt")
sorted_taxa_rank = {k: v for k, v in sorted(taxa_rank.items(), key = lambda item: item[1], reverse=True)}
place = 0
for taxon, rank in sorted_taxa_rank.items():
    place += 1
    species_name = get_species_name_from_taxid(taxon)
    print(f"{place} | {species_name}: {rank}")

1 | Mus musculus: 324
2 | Homo sapiens: 51
3 | Canis lupus familiaris: 44
4 | Rattus norvegicus: 34
5 | Acomys russatus: 26
6 | Mus pahari: 25
7 | Mus caroli: 25
8 | Mastomys coucha: 24
9 | Onychomys torridus: 23
10 | Canis lupus: 21
11 | Canis lupus dingo: 19
12 | Peromyscus maniculatus bairdii: 19
13 | Apodemus sylvaticus: 18
14 | Nyctereutes procyonoides: 15
15 | Physeter catodon: 15
16 | Mesocricetus auratus: 14
17 | Peromyscus leucopus: 14
18 | Equus asinus: 13
19 | Rattus rattus: 13
20 | Arvicanthis niloticus: 13
21 | Peromyscus californicus insignis: 13
22 | Peromyscus eremicus: 12
23 | Heterocephalus glaber: 12
24 | Psammomys obesus: 12
25 | Myodes glareolus: 12
26 | Grammomys surdaster: 11
27 | Cricetulus griseus: 11
28 | Onychomys leucogaster: 10
29 | Halichoerus grypus: 10
30 | Chionomys nivalis: 10
31 | Balaenoptera acutorostrata: 9
32 | Meriones unguiculatus: 9
33 | Microtus oregoni: 9
34 | Microtus fortis: 9
35 | Felis catus: 9
36 | Delphinus delphis: 8
37 | Hyperoodon am